In [12]:
import cv2 as cv
import numpy as np

In [13]:
captured_video = cv.VideoCapture('161.mp4')
print(captured_video)

<VideoCapture 00000209E3B419F0>


In [14]:

def concat_tile(im_list_2d):
    return cv.vconcat([cv.hconcat(im_list_h) for im_list_h in im_list_2d])
MOG = cv.createBackgroundSubtractorMOG2()
KNN = cv.createBackgroundSubtractorKNN()

_, frame_zero = captured_video.read()
frame_zero = cv.resize(frame_zero, (640, 360))

while True:
    retval, frame = captured_video.read()
    if not retval:
        break
    frame = cv.resize(frame, (640, 360))
    MOGmask = MOG.apply(frame)
    #MOGmask = cv.cvtColor(cv.merge([MOGmask,MOGmask,MOGmask]), cv.COLOR_RGB2BGR)
    KNNmask = KNN.apply(frame)
    res = cv.vconcat([MOGmask, MOGmask])
    res_2 = cv.vconcat([cv.bitwise_and(frame, frame_zero, mask=MOGmask), cv.bitwise_and(frame, frame_zero, mask=KNNmask)])
    cv.imshow('mog and knn ', res)
    cv.imshow('bitwise_and mog and knn', res_2)
    cv.imshow('frame',frame )
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

captured_video.release()
cv.destroyAllWindows()

In [17]:
captured_video = cv.VideoCapture('161.mp4')
print(captured_video)

<VideoCapture 00000209E3B41C70>


In [18]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = captured_video.read()
old_frame = cv.resize(old_frame, (640, 360))
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = captured_video.read()
    frame = cv.resize(frame, (640, 360))
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

captured_video.release()
cv.destroyAllWindows()

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import cv2 as cv
import torchvision
import torchvision.utils
from torchvision.models import detection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [20]:
captured_video = cv.VideoCapture('163.mkv')
print(captured_video)

device = torch.device("cuda:0")
#device = torch.device("cpu")

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
model = model.float().to(device)

<VideoCapture 00000209E3B41570>


In [21]:
threshold = 0.5

frame_idx = 0
label = None

while True:
    retval, frame = captured_video.read()
    if not retval:
        break

    frame = cv.resize(frame, (640, 360))

    frame = torch.tensor(np.swapaxes(np.swapaxes(frame, 0, 2),1,2)).float() / 255

    if (frame_idx % 20 == 0 and frame_idx >= 60): # 'and frame_idx >= 50' only for skipping start vieo in my situation
        label = model([frame.to(device)])[0]

    frame =  frame.permute(1, 2, 0).detach().cpu().numpy()

    if (label!=None):
        for i in range(len(label["labels"])):
            if label["scores"][i] < threshold:
                continue
            if label["labels"][i] == 1:
                frame = cv.rectangle(frame, (int(label['boxes'][i][0]), int(label['boxes'][i][1]) ), ( int(label['boxes'][i][2]),int(label['boxes'][i][3])), (255,0,0), 1)

    cv.imshow('frame', frame)

    #print(frame_idx)
    frame_idx +=1

    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

captured_video.release()
cv.destroyAllWindows()

In [ ]:
captured_video.release()
cv.destroyAllWindows()